In [19]:
import pandas as pd
import psycopg2
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
import numpy as np
import seaborn as sns 
import statsmodels.stats.proportion as proportion
from scipy.stats import ttest_ind,mannwhitneyu,shapiro,norm
from statsmodels.stats.weightstats import ztest
from tqdm import tqdm
import timeit
from scipy import stats
import math
from datetime import date, datetime, timedelta
import time
from sqlalchemy import create_engine, text
from sqlalchemy.orm import sessionmaker
import warnings
warnings.filterwarnings("ignore")
import clickhouse_connect  

from credential import postgres_secret,clickhouse_dwh_secret

def get_engine(user):
    if user == postgres_secret['user']:
        db_name = postgres_secret['db_name']
        password = postgres_secret['password']
        host = postgres_secret['host']
        engine = create_engine(f'postgresql://{user}:{password}@{host}:6432/{db_name}')
    elif user == clickhouse_dwh_secret['user']:
            db_name = clickhouse_dwh_secret['db_name'] 
            password = clickhouse_dwh_secret['password']
            host = clickhouse_dwh_secret['host']
            engine = create_engine(f'clickhouse://{user}:{password}@{host}:8123/{db_name}')
    return engine
    
connection_clickhouse = clickhouse_connect.get_client(
    host = clickhouse_dwh_secret['host'],
    port= '8123',
    username = clickhouse_dwh_secret['user'],
    password = clickhouse_dwh_secret['password'],
    database='sandbox'
    )

    
def execute(SQL, user):
    start_time = time.time()  # запоминаем время начала выполнения функции
    engine = get_engine(user)
    Session = sessionmaker(bind=engine)  # sessions factory ()
    with Session() as session: # open session
        result = session.execute(text(SQL))
        df = pd.DataFrame(result.fetchall(), columns=result.keys())
        
    end_time = time.time()  # запоминаем время окончания выполнения функции
    execution_time = round(end_time - start_time,4) # вычисляем время выполнения   
    
    print(f"Время выполнения функции: {execution_time} секунд")
    print()
    return df

In [21]:
import gspread
from gspread_dataframe import get_as_dataframe
from google.oauth2.service_account import Credentials

In [23]:
pwd

'/Users/kemran/Desktop/work_files/python_files'

# Шаг 1: Настройка доступа к Google Sheets API

Для работы с Google Sheets через `pandas` нужно создать **сервисный аккаунт** и получить файл с учётными данными, которые позволят Python-скрипту взаимодействовать с вашим Google Sheets. Следуйте этим шагам:

### 1.1 Создайте проект в Google Cloud Console

1. Перейдите в [Google Cloud Console](https://console.developers.google.com/).
2. Если у вас уже есть проект, можно использовать его. Если нет, нажмите **Select a project** в верхнем меню и выберите **New Project** (Новый проект).
3. Дайте проекту имя, выберите местоположение, и нажмите **Create** (Создать).

### 1.2 Включите Google Sheets API для проекта

1. В левом меню перейдите в **Library** (Библиотека).
2. В строке поиска введите **Google Sheets API** и выберите его из списка.
3. Нажмите **Enable** (Включить) для активации API.

### 1.3 Создайте сервисный аккаунт

1. В меню слева выберите **APIs & Services** → **Credentials** (Учётные данные).
2. Нажмите **Create Credentials** (Создать учётные данные) и выберите **Service Account** (Сервисный аккаунт).
3. Укажите имя аккаунта и описание, затем нажмите **Create and Continue** (Создать и продолжить).
4. На следующем шаге настройте права доступа. В большинстве случаев их можно оставить по умолчанию и нажать **Done**.

После создания сервисного аккаунта вы увидите его в списке на странице Credentials.

### 1.4 Скачайте JSON-файл с учётными данными

1. Найдите созданный сервисный аккаунт в списке, нажмите на его название.
2. Перейдите на вкладку **Keys** (Ключи) и выберите **Add Key** (Добавить ключ) → **Create New Key** (Создать новый ключ).
3. Выберите **JSON** и нажмите **Create** (Создать). Файл будет автоматически загружен на ваш компьютер. Этот JSON-файл будет использоваться в вашем Python-скрипте для аутентификации.

### 1.5 Настройте доступ к вашему Google Sheets документу

1. Откройте Google Sheets, с которым вы хотите работать.
2. Нажмите **Share** (Поделиться) в правом верхнем углу.
3. Вставьте email адрес сервисного аккаунта (он указан в JSON-файле, который вы скачали, и имеет вид `your-service-account@your-project-id.iam.gserviceaccount.com`) и дайте доступ **Editor** (Редактор).
4. Нажмите **Send** (Отправить).

Теперь у вашего сервисного аккаунта есть доступ к этому документу, и Python-скрипт сможет его открывать и редактировать.


In [11]:
# Укажите путь к вашему JSON-файлу с учётными данными
credentials_path = '../python_files/product-analytics-96609-bd42b7d7f66a.json'
scope = ["https://www.googleapis.com/auth/spreadsheets"]

# Создание аутентификации
credentials = Credentials.from_service_account_file(credentials_path, scopes=scope)
gc = gspread.authorize(credentials)

# Откройте Google Sheet по URL или по его ID
spreadsheet_url = "https://https://docs.google.com/spreadsheets/d/1AvO-X0sK34o-WizFlqD-xhBdrTz7PwMT2XW9dQmpoOU/edit?gid=0#gid=0"
spreadsheet = gc.open_by_url(spreadsheet_url)

# Откройте конкретный лист по имени
worksheet = spreadsheet.worksheet("Лист1")

# Импортируйте данные из листа в DataFrame
df = get_as_dataframe(worksheet)

In [12]:
df

,bonus_title_old,bonus_title_new
0,14 дней за 1 рубль. Промостраницы,Яндекс. Промостраницы
1,"35ZA1 Кампания ""Хватит""",Реклама. Все источники
2,45DAYS-уникальные (2),EMAIL
3,Bloggers JUNE Mediacube 2024,Bloggers
4,C2H5OH,"Промокод для ""своих"""
...,...,...
119,Промокод без привязки карты на 12 мес (Армения...,Others
120,Промокод без привязки карты на 12 мес (тест),TEST
121,"Промокод для сторов и моб кампаний, март",Mobile
122,Таттелеком_Тестовые_коды_3мес_10июл2024,TEST


In [15]:
query = '''CREATE TABLE sandbox.mekhtiev_bonus_local ON CLUSTER 'viasat_cluster'
            (
                bonus_title_old String,
                bonus_title_new String
                )
             ENGINE = ReplicatedMergeTree('/clickhouse/tables/{shard}/mekhtiev_bonus_local', '{replica}')
             ORDER BY bonus_title_old
             '''
execute(query,user = 'kmekhtiev')

Время выполнения функции: 1.1083 секунд



,host,port,status,error,num_hosts_remaining,num_hosts_active
0,dwh02,9000.0,253.0,Code: 253. DB::Exception: Replica /clickhouse/...,3.0,0.0
1,dwh03,9000.0,253.0,Code: 253. DB::Exception: Replica /clickhouse/...,2.0,0.0
2,dwh01,9000.0,253.0,Code: 253. DB::Exception: Replica /clickhouse/...,1.0,0.0
3,dwh04,9000.0,253.0,Code: 253. DB::Exception: Replica /clickhouse/...,0.0,0.0
4,Code: 253. DB::Exception: There was an error o...,NaN,NaN,None,NaN,NaN


In [ ]:
query = f'''CREATE TABLE datamarts.mekhtiev_bonus ON CLUSTER 'viasat_cluster'
            (
                bonus_title_old String,
                bonus_title_new String
                )
             
             ENGINE = Distributed(viasat_cluster, datamarts, mekhtiev_bonus_local, rand())
             '''
execute(query,user = 'kmekhtiev')

In [25]:
execute(SQL = f" TRUNCATE TABLE sandbox.mekhtiev_bonus2_local ON CLUSTER 'viasat_cluster' ", user='kmekhtiev')
connection_clickhouse.insert_df('mekhtiev_bonus2', df)

print("Данные залиты")

Время выполнения функции: 0.7528 секунд

Данные залиты
